In [1]:
import os
import re
import urllib.request

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
from rasterio.warp import transform
import pandas as pd
import xarray as xr


In [12]:
ref = xr.open_dataset("../../../data_general/proc_sif-optim/covar_nc/sif_refgrid.nc")
lon, lat = ref.indexes.values()
lat[0]

24.0

In [21]:
%%bash 

cat > target_grid << EOF
gridtype = lonlat
xsize = 13334
ysize = 5778
xfirst = -125.0
xinc = 0.0045
yfirst = 24.0
yinc = 0.0045
EOF

In [22]:
%%bash
cdo griddes $src > mygrid
sed -i "s/generic/lonlat/g" mygrid


cdo (Abort): Too few streams specified! Operator griddes needs 1 input stream and 0 output stream!


In [23]:
%%bash 

cdo setgrid,mygrid $src infile_fixedgrid.nc


cdo (Abort): Too few streams specified! Operator setgrid,mygrid needs 1 input stream and 1 output stream!


CalledProcessError: Command 'b'\ncdo setgrid,mygrid $src infile_fixedgrid.nc\n'' returned non-zero exit status 1.

In [17]:
%%bash

cdo griddes $src > mygrid
sed -i "s/generic/lonlat/g" mygrid


cdo (Abort): Too few streams specified! Operator griddes needs 1 input stream and 0 output stream!


In [15]:
%%bash 
src="../../../data_general/proc_sif-optim/myd_lst/myd11a1_2018.nc"
cdo -f nc4c -z zip_6 remapbil,mygrid $src test.nc


cdo    remapbil (Abort): Unsupported target grid type (generic)!


CalledProcessError: Command 'b'src="../../../data_general/proc_sif-optim/myd_lst/myd11a1_2018.nc"\ncdo -f nc4c -z zip_6 remapbil,mygrid $src test.nc\n'' returned non-zero exit status 1.

In [4]:
%%bash
gdalinfo "../../../data_general/proc_sif-optim/sif_refgrid.tif"

Driver: GTiff/GeoTIFF
Files: ../../../data_general/proc_sif-optim/sif_refgrid.tif
Size is 13334, 5778
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (-125.002250000000004,49.998750000000001)
Pixel Size = (0.004499999999993,-0.004500000000000)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (-125.0022500,  49.9987500) (125d 0' 8.10"W, 49d59'55.50"N)
Lower Left  (-125.0022500,  23.9977500) (125d 0' 8.10"W, 23d59'51

In [ ]:
%%bash
gdalinfo "../../../data_general/proc_sif-optim/myd_lst/myd11a1_2018.nc"../../../data_general/proc_sif-optim/covar_nc/

In [5]:
%%bash 
src="../../../data_general/proc_sif-optim/myd_lst/myd11a1_2018.nc"
ref="../../../data_general/proc_sif-optim/sif_refgrid.tif"

gdal_translate -r nearest -tr 0.0045 0.0045 $src test.nc

Process is interrupted.


In [ ]:
%%bash 
src="../../../data_general/proc_sif-optim/myd_lst/myd11a1_2018.nc"
ref="../../../data_general/proc_sif-optim/covar_nc/sif_refgrid.nc"
cdo -f nc4c -z zip_6 remap